In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/grocery-store-dataset/GroceryDataset.csv


In [2]:
data = pd.read_csv("/kaggle/input/grocery-store-dataset/GroceryDataset.csv")
data.sample(10)

,Sub Category,Price,Discount,Rating,Title,Currency,Feature,Product Description
795,Household,$17.99,No Discount,NaN,"Post-it Notes, Canary Yellow, 3"" x 3"" 100 Shee...",$,NaN,Model: MMM202624YWSelf Stick NotesCanary Yello...
28,Bakery & Desserts,$34.99,No Discount,Rated 4.5 out of 5 stars based on 702 reviews.,Ferrara's Bakery Rainbow Cookies 1.5 lb,$,Tricolor Rainbow Cookies Each cookie is appr...,Ferrara's Bakery of Little Italy in New York C...
1404,Paper & Plastic Products,$28.99,No Discount,NaN,"Polyvinyl Films All Purpose Food Wrap, 18 in x...",$,More Cling More Stretch More Strength Ma...,Wrap BowlsCover TraysGreat InsulatorUse in Mic...
1219,Pantry & Dry Goods,$10.99,No Discount,NaN,"Prego, Pasta Sauce Low Sodium 45 oz, 3-Count",$,Italian Sauce Traditional Lower Sodium Glu...,25% Lower Sodium Than Prego Traditional\n40% o...
1506,Snacks,$9.99,No Discount,NaN,"Lance Toasty Cracker Sandwiches, 1.29 oz, 40-c...",$,Real Peanut Butter No High-Fructose Corn Syr...,1.29 oz Packs
129,Beverages & Water,$74.99,No Discount,Rated 4.3 out of 5 stars based on 354 reviews.,"Pressed Cold-Pressed Greens, Roots & Citrus Bu...",$,"12 Oz. Bottles\nNo Sugars Added, No Added Wate...","Due to the perishable nature of this item, ord..."
15,Bakery & Desserts,$59.99,No Discount,Rated 4.5 out of 5 stars based on 452 reviews.,"David’s Cookies Variety Cheesecakes, 2-pack (2...",$,"""10"""" Cheesecakes, 2-count\n1 Turtle Style, Ch...",We've done it again with our Turtle Cheesecake...
843,Kirkland Signature Grocery,$52.99,No Discount,Rated 4.6 out of 5 stars based on 3871 reviews.,Kirkland Signature Nature's Domain Salmon & Sw...,$,Salmon & Sweet Potato Formula\nGrain-Free\n35 ...,Item may be available in your local warehouse ...
558,Deli,$249.99,No Discount,Rated 4.2 out of 5 stars based on 42 reviews.,Tsar Nicoulai Caviar Tasting Flight Gift Set,$,Vacuum sealed Caviar jar keeps 60 days unopene...,Enjoy a caviar tasting experience inspired by ...
515,Coffee,$16.99,No Discount,NaN,"Nestlé Coffee-mate Coffee Creamer, Hazelnut, P...",$,1.5-liter Pump Bottle full of Liquid Creamer ...,The nutty Hazelnut flavored Nestlé Coffee-mate...


1. Check for Missing Value
1. Fix datatype
1. encode categorical data
1. change rating column to something numerical value
1. Fetch quantiy from title
1. check out differnt currency to purchases

In [3]:
data.shape

(1757, 8)

## 1. Check and Fix Missing Value
> Here, I can see Price column has 3 missing value, Rating has 1075 missing value, currency has 5, Feature has 18 and Product Discription has 42 missing value.

In [4]:
data.isnull().sum()

Sub Category              0
Price                     3
Discount                  0
Rating                 1075
Title                     0
Currency                  5
Feature                  18
Product Description      42
dtype: int64

### 1.a). Find missing value percentage.
- if any column has more then 60 percentage of data is missing value then it is good idea to remove that columns
- Missing Value Percentage = (Total Number of Values/Number of Missing Values)×100

In [5]:
def find_percentage(columns_name):
    missing_cnt = columns_name.isnull().sum()
    total_cnt = columns_name.shape
    missing_percentage = ((missing_cnt/total_cnt)*100).round(2)
    return missing_percentage

In [6]:
data.apply(find_percentage)

,Sub Category,Price,Discount,Rating,Title,Currency,Feature,Product Description
0,0.0,0.17,0.0,61.18,0.0,0.28,1.02,2.39


### 1.b). Drop Rating Column 

In [7]:
#since Rating has 61.18 percentage of null value so drop the columns
data.drop(['Rating'], axis=1, inplace=True)

### 1.c). Handle Missing Value
1. Price
1. Currency
1. Feature
1. Product Description

In [8]:
data['Price'] = data['Price'].str.replace('[^\d.]', '', regex=True)
data['Price'] = pd.to_numeric(data['Price'], errors='coerce')
data['Price'] = data['Price'].round(2)


In [9]:
data[data['Price'].isnull()]

,Sub Category,Price,Discount,Title,Currency,Feature,Product Description
389,Cleaning Supplies,NaN,No Discount,"simplehuman Custom Fit Liners, 300-pack",$,Perfectly Fits simplehuman Sensor and Step Can...,Limit 5 per memberOur custom fit liners are de...
578,Floral,NaN,No Discount,Inspire Floral Arrangement,$,Farm Fresh Flowers Pick Your Arrival Date at...,"Let this great combination of flowers, colors ..."
579,Floral,NaN,No Discount,Tranquility Vase Arrangement,$,Farm Fresh Flowers Pick Your Arrival Date at...,Let your mind drift away into tranquil thought...
581,Floral,NaN,No Discount,Passion Vase Arrangement,$,Farm Fresh Flowers Pick Your Arrival Date at...,"Fuel your Passion, Find Your Passion, Follow y..."
589,Floral,NaN,No Discount,Day Dream Vase Arrangement,$,Farm Fresh Flowers Pick Your Arrival Date at...,Don't quit your daydreams as they bring joy to...
611,Floral,NaN,No Discount,Valentine's Day Pre-Order Endless Love,$,Pick Your Arrival Date at Checkout Gift Mess...,"Artistically designed and hand-tied, the Endle..."
630,Floral,NaN,No Discount,Valentine's Day Perfect Love,$,Pick Your Arrival Date at Checkout Gift Mess...,"Artistically designed and hand-tied, the Perfe..."
633,Floral,NaN,No Discount,Soft Grace Wedding Collection,$,"Pre-Order Months in Advance 4-Piece, 10-Piec...",Costco.com offers a magnificent assortment of ...
635,Floral,NaN,No Discount,Wedding Variety Collection,$,Pre-Order Months in Advance 6-Piece or 10-Pi...,"6-piece Candle Rings Includes:\n• Six (6) 12"" ..."
639,Floral,NaN,No Discount,Orange Sorbet Wedding Package,$,"Pre-Order Months in Advance 4-Piece, 10-Piec...",Costco.com offers a magnificent assortment of ...


In [10]:
data['Price'].isnull().sum()

17

In [11]:
df = data.dropna()
df.shape

(1682, 7)

In [12]:
df.isnull().sum()

Sub Category           0
Price                  0
Discount               0
Title                  0
Currency               0
Feature                0
Product Description    0
dtype: int64

### Display Unique Sub Category

In [13]:
print(data['Sub Category'].unique())
print("\nTotal Sub Category count =", data['Sub Category'].nunique())

['Bakery & Desserts' 'Beverages & Water' 'Breakfast' 'Candy'
 'Cleaning Supplies' 'Coffee' 'Deli' 'Floral' 'Gift Baskets' 'Household'
 'Kirkland Signature Grocery' 'Laundry Detergent & Supplies'
 'Meat & Seafood' 'Organic' 'Pantry & Dry Goods'
 'Paper & Plastic Products' 'Poultry' 'Seafood' 'Snacks']

Total Sub Category count = 19


In [14]:
# Example number
number = 3.456

# Round to two decimal places
rounded_number = round(number, 2)

# Print the result
print("Original number:", number)
print("Rounded number:", rounded_number)

Original number: 3.456
Rounded number: 3.46
